In [1]:
import pickle
import os

import doc2vec
import scipy.spatial.distance

import rank_metrics
import numpy as np

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # cancel optimization complaints by tensorflow

In [3]:
raw_data_folder = 'aminder_org_v1'
rel_labels_fname = 'relevance_labels_' + raw_data_folder + '.p'
doc2vec_model_folder = 'd2v_model'

distance_measure = 'cosine'

In [4]:
"""
Get relevance labels
"""

if not os.path.isfile(rel_labels_fname):
    print('Computing relevance labels using bm25... (this takes a while, but should only be done once..)')
    import prepare_relevance_labels

    prepare_relevance_labels.prepare_relevance_labels(output_fname=rel_labels_fname, folder=raw_data_folder)

with open(rel_labels_fname, 'rb') as f:
    source_dict, docs, doc_names, tokenized, bm25_scores, sorted_bm25_indices = pickle.load(f)

Computing relevance labels using bm25... (this takes a while, but should only be done once..)
111000
Computing BM25...
Done computing bm25, compute average IDF...
Done computing average IDF.
Computing relative scores...
0 / 111000
1000 / 111000
2000 / 111000
3000 / 111000
4000 / 111000
5000 / 111000


KeyboardInterrupt: 

In [ ]:
"""
Train paragraph vectors
"""

# Always train, because restoring does not work
if not False:  # os.path.exists(doc2vec_model_folder):
    print('Initializing and training paragraph vectors')
    d2v = doc2vec.Doc2Vec(batch_size=128,
                          window_size=8,
                          concat=True,
                          architecture='pvdm',
                          embedding_size_w=300,  # word embedding size
                          embedding_size_d=300,  # document embeding size
                          vocabulary_size=50000,
                          document_size=len(docs),
                          loss_type='sampled_softmax_loss',
                          n_neg_samples=64,
                          optimize='Adagrad',
                          learning_rate=1.0,
                          n_steps=100001  # 100001
                          )

    d2v.fit(tokenized)
    latest_checkpoint = d2v.save(doc2vec_model_folder)
else:
    d2v = doc2vec.Doc2Vec.restore(doc2vec_model_folder)

In [ ]:
d2v = doc2vec.Doc2Vec.restore(doc2vec_model_folder)

In [ ]:
"""
Evaluate
"""
distances = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(d2v.doc_embeddings, metric=distance_measure))

results = []
hits = []
sorted_d2v_distance_indices = []
for doc_index, distance in enumerate(distances):
    sorted_distance_indices = sorted(range(len(distance)), key=lambda x: distance[x], reverse=False)
    relevance_set = set(sorted_bm25_indices[doc_index][:10])
    hit = np.array([ix in relevance_set for ix in sorted_distance_indices[:10]], dtype=int)
    average_precision = rank_metrics.average_precision(hit)
    ndcg_at_10 = rank_metrics.ndcg_at_k(hit, 10)
    sorted_d2v_distance_indices.append(sorted_distance_indices)    
    hits.append(hit)
    results.append({
        'average_precision': average_precision,
        'ndcg_at_10': ndcg_at_10,
    })

print("MAP: ", rank_metrics.mean_average_precision(hits))
print("MRR: ", rank_metrics.mean_reciprocal_rank(hits))

NDIMS = 64

MAP:  0.646
MRR:  0.711

NDIMS = 128

MAP:  0.651
MRR:  0.707

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(results)
print(df)

In [ ]:
def print_result_difference(ix, number_of_articles=2):
    print("Source document:")
    print(docs[ix], '\nMost similar (BM25) documents:\n' + 50*'-')
    for i in sorted_bm25_indices[ix][:number_of_articles]:
        print(i)
        print(docs[i][:300] + '...')
        print(25*'-')
    print('\nClosest Doc2Vec documents:\n' + 50*'-')
    for i in sorted_d2v_distance_indices[ix][:number_of_articles]:
        print(i)
        print(docs[i][:300] + '....')
        print(25*'-')
print_result_difference(300)